In [1]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from collections import Counter

In [39]:
link = "https://github.com/maver1ck1209/Hackrx-HealthGenie/blob/main/doctor%20recommendation/data/Original_Dataset.csv"
print(link.replace("github.com", "raw.githubusercontent.com").replace("/blob/", "/"))

https://raw.githubusercontent.com/maver1ck1209/Hackrx-HealthGenie/main/doctor%20recommendation/data/Original_Dataset.csv


In [2]:
sd_data = pd.read_csv("https://raw.githubusercontent.com/maver1ck1209/Hackrx-HealthGenie/main/doctor%20recommendation/data/Original_Dataset.csv",  index_col=None)

In [3]:
sc = np.array(sd_data.columns)
sc = sc[1:]

In [4]:
sym = np.array(sd_data.iloc[:, 1:].values.flatten())
sym = list(set(sym))
print(sym)

[nan, ' skin_rash', ' chills', ' extra_marital_contacts', ' high_fever', ' lack_of_concentration', ' acidity', ' spotting_ urination', ' nodal_skin_eruptions', ' dark_urine', ' visual_disturbances', ' sunken_eyes', ' throat_irritation', ' sweating', ' swollen_extremeties', ' distention_of_abdomen', ' knee_pain', ' silver_like_dusting', ' pain_in_anal_region', ' nausea', ' depression', ' abdominal_pain', ' mucoid_sputum', ' runny_nose', ' loss_of_appetite', ' yellow_crust_ooze', ' slurred_speech', ' headache', ' rusty_sputum', ' hip_joint_pain', ' weakness_of_one_body_side', ' neck_pain', ' continuous_sneezing', ' blister', ' family_history', ' cough', ' yellowish_skin', ' prominent_veins_on_calf', ' muscle_wasting', ' weakness_in_limbs', ' vomiting', ' congestion', ' yellow_urine', ' mood_swings', ' shivering', ' swelled_lymph_nodes', ' stomach_pain', ' phlegm', ' cramps', ' muscle_pain', ' malaise', ' blackheads', ' swollen_blood_vessels', ' scurring', ' inflammatory_nails', ' weight_

In [5]:
for s in sym:
    sd_data[s] = sd_data.iloc[:, 1:].apply(lambda x: int(s in x.values), axis=1)

sd_data_1 = sd_data.drop(columns=sc)
sd_data_1.head()

,Disease,NaN,skin_rash,chills,extra_marital_contacts,high_fever,lack_of_concentration,acidity,spotting_ urination,nodal_skin_eruptions,...,dizziness,pain_behind_the_eyes,excessive_hunger,fatigue,weight_loss,brittle_nails,cold_hands_and_feets,receiving_blood_transfusion,swelling_of_stomach,watering_from_eyes
0,Fungal infection,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Fungal infection,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Fungal infection,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Fungal infection,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Fungal infection,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [6]:
sd_data_1 = sd_data_1.loc[:, sd_data_1.columns.notna()]
sd_data_1.head()

,Disease,skin_rash,chills,extra_marital_contacts,high_fever,lack_of_concentration,acidity,spotting_ urination,nodal_skin_eruptions,dark_urine,...,dizziness,pain_behind_the_eyes,excessive_hunger,fatigue,weight_loss,brittle_nails,cold_hands_and_feets,receiving_blood_transfusion,swelling_of_stomach,watering_from_eyes
0,Fungal infection,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Fungal infection,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Fungal infection,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Fungal infection,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Fungal infection,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
sd_data_2 = sd_data_1.drop(columns='Disease')


In [8]:
sd_data_2.loc['total']= sd_data_2.sum()

In [9]:
avg = sd_data_2.iloc[-1].sum()
avg = avg//131
l_sym = []
for column in sd_data_2.columns:
    if(sd_data_2.loc['total',column] > avg):
        l_sym.append(column)
l_sym.append('Disease')
print(len(l_sym))

32


In [10]:
sd_data_3 = sd_data_1.drop(columns=[col for col in sd_data_1.columns if col not in l_sym])

In [50]:
#label_encoding the disease attribute
# var_mod = ['Disease']
# le = LabelEncoder()
# for i in var_mod:
#     sd_data_3[i] = le.fit_transform(sd_data_3[i])

In [52]:
# np.save('classes.npy', le.classes_)

In [11]:
X = sd_data_3.drop(columns="Disease")
y = sd_data_3["Disease"]

In [12]:
#SVM
model = svm.SVC(probability=True)
model.fit(X, y)
Pkl_Filename = "model/SVM_Model.pkl"

with open(Pkl_Filename, 'wb') as file:
    pickle.dump(model, file)

In [13]:
#Random Forest
model1 = RandomForestClassifier()
model1.fit(X, y)
Pkl_Filename = "model/Random_Forest_Model.pkl"

with open(Pkl_Filename, 'wb') as file:
    pickle.dump(model1, file)

In [14]:
#KNN
model2 = KNeighborsClassifier()
model2.fit(X, y)
Pkl_Filename = "model/KNN_Model.pkl"

with open(Pkl_Filename, 'wb') as file:
    pickle.dump(model2, file)

In [15]:
#Naive Bayes
model3 = GaussianNB()
model3.fit(X, y)
Pkl_Filename = "model/Naive_Bayes_Model.pkl"

with open(Pkl_Filename, 'wb') as file:
    pickle.dump(model3, file)

In [16]:
from pydantic import BaseModel

class Symptom(BaseModel):
    loss_of_balance: int
    chest_pain: int
    dark_urine: int
    sweating: int
    blurred_and_distorted_vision: int
    abdominal_pain: int
    joint_pain: int
    fatigue: int
    yellowish_skin: int
    yellowing_of_eyes: int
    phlegm: int
    loss_of_appetite: int
    diarrhoea: int
    malaise: int
    cough: int
    high_fever: int
    mild_fever: int
    vomiting: int
    breathlessness: int
    swelled_lymph_nodes: int
    headache: int
    weight_loss: int
    muscle_pain: int
    nausea: int
    irritability: int
    dizziness: int
    lethargy: int
    skin_rash: int
    itching: int
    excessive_hunger: int
    chills: int
    class Config:
        schema_extra = {
            "example": {
                "loss_of_balance": 0,
                "chest_pain": 1,
                "dark_urine": 0,
                "sweating": 1,
                "blurred_and_distorted_vision": 0,
                "abdominal_pain": 1,
                "joint_pain": 0,
                "fatigue": 0,
                "yellowish_skin": 0,
                "yellowing_of_eyes": 0,
                "phlegm": 0,
                "loss_of_appetite": 0,
                "diarrhoea": 0,
                "malaise": 0,
                "cough": 0,
                "high_fever": 0,
                "mild_fever": 0,
                "vomiting": 0,
                "breathlessness": 0,
                "swelled_lymph_nodes": 0,
                "headache": 0,
                "weight_loss": 0,
                "muscle_pain": 0,
                "nausea": 0,
                "irritability": 0,
                "dizziness": 0,
                "lethargy": 0,
                "skin_rash": 0,
                "itching": 0,
                "excessive_hunger": 0,
                "chills": 0,
            }
        }

In [18]:
from fastapi import FastAPI
import pickle

app = FastAPI()

@app.on_event("startup")
def load_model():
    global model, model1, model2, model3, encoder
    encoder = LabelEncoder()
    # encoder.classes_ = np.load('model/classes.npy', allow_pickle=True)
    model = pickle.load(open("model/KNN_Model.pkl", "rb"))
    model1 = pickle.load(open("model/Naive_Bayes_Model.pkl", "rb"))
    model2 = pickle.load(open("model/Random_Forest_Model.pkl", "rb"))
    model3 = pickle.load(open("model/SVM_Model.pkl", "rb"))

@app.get('/')
def index():
    return {'message': 'This is the homepage of the API '}


@app.post('/predict')
def get_music_category(data: Symptom):
    received = data.dict()
    loss_of_balance = received['loss_of_balance']
    chest_pain = received['chest_pain']
    dark_urine = received['dark_urine']
    sweating = received['sweating']
    blurred_and_distorted_vision = received['blurred_and_distorted_vision']
    abdominal_pain = received['abdominal_pain']
    joint_pain = received['joint_pain']
    fatigue = received['fatigue']
    yellowish_skin = received['yellowish_skin']
    yellowing_of_eyes = received['yellowing_of_eyes']
    phlegm = received['phlegm']
    loss_of_appetite = received['loss_of_appetite']
    diarrhoea = received['diarrhoea']
    malaise = received['malaise']
    cough = received['cough']
    high_fever = received['high_fever']
    mild_fever = received['mild_fever']
    vomiting = received['vomiting']
    breathlessness = received['breathlessness']
    swelled_lymph_nodes = received['swelled_lymph_nodes']
    headache = received['headache']
    weight_loss = received['weight_loss']
    muscle_pain = received['muscle_pain']
    nausea = received['nausea']
    irritability = received['irritability']
    dizziness = received['dizziness']
    lethargy = received['lethargy']
    skin_rash = received['skin_rash']
    itching = received['itching']
    excessive_hunger = received['excessive_hunger']
    chills = received['chills']
    dis_name = []
    pred_name = model.predict([[loss_of_balance, chest_pain, dark_urine, sweating, blurred_and_distorted_vision, abdominal_pain, joint_pain,
                                fatigue, yellowish_skin, yellowing_of_eyes, phlegm, loss_of_appetite, diarrhoea, malaise, cough,
                                high_fever, mild_fever, vomiting, breathlessness, swelled_lymph_nodes, headache, weight_loss, muscle_pain,
                                nausea, irritability, dizziness, lethargy, skin_rash, itching, excessive_hunger, chills]]).tolist()[0]
    # dis_name.append(le.inverse_transform(pred_name))
    dis_name.append(pred_name)
    pred_name = model1.predict([[loss_of_balance, chest_pain, dark_urine, sweating, blurred_and_distorted_vision, abdominal_pain, joint_pain,
                                fatigue, yellowish_skin, yellowing_of_eyes, phlegm, loss_of_appetite, diarrhoea, malaise, cough,
                                high_fever, mild_fever, vomiting, breathlessness, swelled_lymph_nodes, headache, weight_loss, muscle_pain,
                                nausea, irritability, dizziness, lethargy, skin_rash, itching, excessive_hunger, chills]]).tolist()[0]
    dis_name.append(pred_name)
    pred_name = model2.predict([[loss_of_balance, chest_pain, dark_urine, sweating, blurred_and_distorted_vision, abdominal_pain, joint_pain,
                                fatigue, yellowish_skin, yellowing_of_eyes, phlegm, loss_of_appetite, diarrhoea, malaise, cough,
                                high_fever, mild_fever, vomiting, breathlessness, swelled_lymph_nodes, headache, weight_loss, muscle_pain,
                                nausea, irritability, dizziness, lethargy, skin_rash, itching, excessive_hunger, chills]]).tolist()[0]
    dis_name.append(pred_name)
    pred_name = model3.predict([[loss_of_balance, chest_pain, dark_urine, sweating, blurred_and_distorted_vision, abdominal_pain, joint_pain,
                                fatigue, yellowish_skin, yellowing_of_eyes, phlegm, loss_of_appetite, diarrhoea, malaise, cough,
                                high_fever, mild_fever, vomiting, breathlessness, swelled_lymph_nodes, headache, weight_loss, muscle_pain,
                                nausea, irritability, dizziness, lethargy, skin_rash, itching, excessive_hunger, chills]]).tolist()[0]
    dis_name.append(pred_name)
    return {'prediction': dis_name}

In [19]:
from colabcode import ColabCode
server = ColabCode(port=10000, code=False)

In [20]:
server.run_app(app=app)

t=2023-07-21T18:11:38+0530 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=C:\\Users\\kiran\\AppData\\Local/ngrok/ngrok.yml legacy_path=C:\\Users\\kiran\\.ngrok2\\ngrok.yml
INFO:     Started server process [11756]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://aa29-103-68-38-66.ngrok-free.app" -> "http://localhost:10000"
INFO:     103.68.38.66:0 - "GET / HTTP/1.1" 200 OK
INFO:     103.68.38.66:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     103.68.38.66:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     103.68.38.66:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     103.68.38.66:0 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [11756]
